# UK Covid-19 Vaccination Dashboard

This notebook is an interactive dashboard showing the uptake of Covid-19 vaccinations in the UK. Data was obtained from Public Health England's [Covid-19 data API](https://coronavirus.data.gov.uk/details/developers-guide).

## Set-up

The data initially displayed on this dashboard was last updated on 3 December. To refresh the dashboard, you can click on the 'Fetch latest data' button below.

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("vax_percentage.json", "rt") as INFILE:
    vax_percentage=json.load(INFILE)
with open("vax_percentage_nation.json", "rt") as INFILE:
    vax_percentage_nation=json.load(INFILE)

In [4]:
def wrangle_data(rawdata):
    # note: rawdata is the json file that we've loaded
    # convert to pandas df
    df = pd.DataFrame.from_dict(rawdata['data'])

    # convert date column into datetime 
    df['date'] = pd.to_datetime(df['date'])
    
    # set date column as index
    df.set_index('date', inplace=True)
    
    # sort the index
    df.sort_index(inplace=True)
    
    return df

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df_vax_p = wrangle_data(vax_percentage)
df_vax_p_nation = wrangle_data(vax_percentage_nation)

In [5]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_vax_percentage_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
        'areaType=overview'
    ]


    # values here are the names of the PHE metrics
    structure = {
        "date": "date",
        "cum1stDosePercentage": "cumVaccinationFirstDoseUptakeByPublishDatePercentage",
        "cum2ndDosePercentage": "cumVaccinationSecondDoseUptakeByPublishDatePercentage",
        "cum3rdDosePercentage": "cumVaccinationThirdInjectionUptakeByPublishDatePercentage"
    }

    vax_percentage_api = Cov19API(filters=filters, structure=structure)
    vax_percentage=vax_percentage_api.get_json()
    return vax_percentage # return data read from the API

# api for nation-specific data
def access_vax_percentage_nation_api():
    filters = [
        'areaType=nation'
    ]

    # values here are the names of the PHE metrics
    structure = {
        "date": "date",
        "nation": "areaName",
        "cum1stDosePercentage": "cumVaccinationFirstDoseUptakeByPublishDatePercentage",
        "cum2ndDosePercentage": "cumVaccinationSecondDoseUptakeByPublishDatePercentage",
        "cum3rdDosePercentage": "cumVaccinationThirdInjectionUptakeByPublishDatePercentage"
    }

    vax_percentage_nation_api = Cov19API(filters=filters, structure=structure)
    vax_percentage_nation = vax_percentage_nation_api.get_json()
    return vax_percentage_nation

In [6]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button

from ipywidgets import Button, Layout

b = Button(layout=Layout(width='30%', height='40px'))

def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    # try except block
    try:
        
        vax_p_data = access_vax_percentage_api()
        vax_p_nation_data = access_vax_percentage_nation_api()
    except:
        apibutton.description = "API unavailable"
    # wrangle the data and overwrite the dataframe for plotting
    else:
        global df_vax_p
        df_vax_p = wrangle_data(vax_p_data)
        global df_vax_p_nation
        df_vax_p_nation = wrangle_data(vax_p_nation_data)
        
        
        # refresh the graphs
        refresh_percentage_graph()
        refresh_nations_graph()
        
        # disabled the button
        apibutton.icon="check"
        apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Fetch latest data', 
    layout = b.layout,
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Access the API to download updated data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download',
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(description='Fetch latest data', icon='download', layout=Layout(height='40px', width='30%'), style=Butt…

## Vaccination uptake in the UK

The Covid-19 vaccination programme in the UK began on 8 December 2020 (but data from the API is only available from 11 Janurary 2020).

All vaccines are given as at least 2 doses, at least 21 days apart. Some people at higher risk from COVID-19 will also receive 3rd doses or booster vaccines.

For more detailed information on how vaccination data is reported, refer to the [documentation](https://coronavirus.data.gov.uk/metrics/doc/cumVaccinationFirstDoseUptakeByPublishDatePercentage).

### UK vaccination uptake
This graph shows the cumulative percentage of the population who have received vaccination dose(s). You can select more than more dose number to compare the uptake of the doses using CTRL-click.

In [10]:
import matplotlib.dates as mdates
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
from ipywidgets import VBox, Label, Layout

series=wdg.SelectMultiple(
    options=['cum1stDosePercentage', 'cum2ndDosePercentage', 'cum3rdDosePercentage'], # options available
    value=['cum1stDosePercentage', 'cum2ndDosePercentage', 'cum3rdDosePercentage'], # initial value
    rows=3, # rows of the selection box
    description='Dose number',
    disabled=False
)

def percentage_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        ax = df_vax_p.plot(y=list(graphcolumns), figsize=(10, 5), title="Vaccination uptake in the UK") # graphcolumns is a tuple - we need a list
        
        # format dates on x axis
        ax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b-%y'))
        ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
        
        # set y-axis limits
        plt.ylim(0,100)
        ax.yaxis.set_minor_locator(AutoMinorLocator(4)) # shows tick for every 5, 20/4 = 5
        
        # add axis labels
        plt.ylabel('Percentage uptake (%)')
        plt.xlabel('Date')
        
        # shorten legend labels
        # ax.legend(labels=['1st dose', '2nd dose', '3rd dose'])
        
        plt.show() # important - graphs won't update properly if this is missing
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# CHANGE this to own function, define another one for the nation graph?
def refresh_percentage_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=series.value
    if current==series.value:
        other=series.options[:2]
    else:
        other=series.options
    series.value=other # forces the redraw
    series.value=current # now we can change it back        
        

# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
graph=wdg.interactive_output(percentage_graph, {'graphcolumns': series})

ctrls=wdg.VBox([series], layout=Layout(align_items='flex-start', justify_content='center'))
# put the graph and the controls side by side
form=wdg.HBox([graph, ctrls])

display(form)

### Vaccination uptake by nation
This graph shows the cumulative percentage of the population in each nation who have received vaccination dose(s) since 11 January 2021. You can toggle between the buttons to view the graphs for each dose number.

In [8]:
# nation-specific vaccination uptake
dose_no=wdg.RadioButtons(
    options=['cum1stDosePercentage', 'cum2ndDosePercentage', 'cum3rdDosePercentage'],
    value='cum1stDosePercentage', 
    layout={'width': 'max-content'}, # If the items' names are long
    description='Dose: ',
    disabled=False
)


def nations_graph(dose_no):  
    df_vax_p_nation.groupby('nation').plot(kind='line', y=dose_no, ax=plt.gca(), figsize=(10, 5), title="Vaccination uptake in the UK by nation")

    # set y-axis limits
    plt.ylim(0,100)

    # add axis labels
    plt.ylabel('Percentage uptake (%)')
    plt.xlabel('Date')
    
    plt.legend(df_vax_p_nation['nation'], loc='upper left')
    
    plt.show() # important - graphs won't update if this is missing 


def refresh_nations_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=dose_no.value
    if current==dose_no.options[0]:
        other=dose_no.options[1]
    else:
        other=dose_no.options[0]
    dose_no.value=other # forces the redraw
    dose_no.value=current # now we can change it back      

# capture output in widget graph   
graph=wdg.interactive_output(nations_graph, {'dose_no': dose_no})

display(dose_no, graph)

RadioButtons(description='Dose: ', layout=Layout(width='max-content'), options=('cum1stDosePercentage', 'cum2n…

Output()

**2021 Fang Ning Tan**  [(Github)](https://github.com/fangningtan/). *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).* 

Note: This dashboard was based on the [DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.